In [2]:
print(df_transaction.shape)
df_transaction.tail(2)

(599961, 9)


,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
599959,35311,2839,2,B03,0339,20190927,14:09,4500,1
599960,35311,2839,6,B03,0324,20190927,14:09,6800,1


In [3]:
print(df_product.shape)
df_product.tail(2)

(1667, 4)


,pd_c,clac_nm1,clac_nm2,clac_nm3
1665,1666,Women's Clothing,Women's Upper Bodywear / Tops,Women's T-shirts / Tops
1666,1667,Women's Clothing,Women's Upper Bodywear / Tops,Women's Vests


# 필요한 데이터만 데이터 프레임 만들기

- 클라이언트 ID, 아이템 ID, buy count, timestamp
    - 거래정보 : 클러이언트 ID, 상품소분류코드, 구매일자, 구매시각, 구매수량
        - **상품소분류코드에서 'unknown'값이 17652있음 -> drop**
    - 상품분류정보 : 상품소분류 코드, 상품소분류명

In [4]:
df = df_transaction[['clnt_id', 'pd_c', 'de_dt', 'buy_ct']]
df.drop(df.loc[df['pd_c']=='unknown'].index, inplace=True)
df['pd_c'] = df['pd_c'].astype(int)
df['de_dt'] = pd.to_datetime(df['de_dt'].astype(str), format='%Y%m%d')
# lookup_product = df_product[['pd_c', 'clac_nm3']]
# df = pd.merge(transaction_df, product_df, how = 'left')

In [5]:
df.shape

(582309, 4)

In [6]:
# lookup frame so we can get the pd_c names back in
lookup_product = df_product[['pd_c', 'clac_nm3']]
lookup_product.drop_duplicates().tail(20)

,pd_c,clac_nm3
1647,1648,Women's Pants
1648,1649,Women's Skirts
1649,1650,Women's Coats
1650,1651,Women's Jackets
1651,1652,Women's Jumpers
1652,1653,Women's Paddingwear
1653,1654,Women's Trench Coats
1654,1655,Women's Fur Coats
1655,1656,Women's Leather Clothing
1656,1657,Maternity Clothing


In [7]:
num_users = df.clnt_id.unique().shape[0]
num_items = df.pd_c.unique().shape[0]
sparsity = 1 - len(df['clnt_id']) / (num_users * num_items)
print('number of users : %d, number of items: %d.' % (num_users, num_items))
print('matrix sparsity : %f' % sparsity)
print(df.head(5))

number of users : 11271, number of items: 1667.
matrix sparsity : 0.969008
   clnt_id  pd_c      de_dt  buy_ct
4    39423   565 2019-09-20       1
5    21279   565 2019-09-20       1
6    48969   572 2019-09-20       1
7    30533   670 2019-09-20       1
8    64346   543 2019-09-20       1


In [8]:
df['buy_ct'].value_counts().sort_index()
df.quantile(.99)

clnt_id    71647.0
pd_c        1628.0
buy_ct         8.0
Name: 0.99, dtype: float64

# Splitting the dataset

In [9]:
def split_data(data, num_users, num_items, split_mode="random", test_ratio=0.1):
    """Split the dataset in random mode or seq-aware mode."""
    if split_mode == "seq-aware":
        train_items, test_itesm, train_list = {}, {}, []
        for line in data.itertuples():
            u, i, rating, time = line[1], line[2], line[3], line[4]
            train_items.setdefault(u, []).append((u, i, rating, time))
            if u not in test_items or test_items[u][-1] < time:
                test_items[u] = (i, rating, time)
        for u in range(1, num_users + 1):
            train_list.extend(sorted(train_items[u], key=lambda k : k[3]))
        test_data = [(key, *value) for key, value in test_items.items()]
        train_data = [item for item in train_list if item not in test_data]
        train_data = pd.DataFrame(train_data)
        test_data = pd.DataFrame(test_data)
    else:
        mask = [True if x == 1 else False for x in np.random.uniform(0, 1, (len(data))) < 1 - test_ratio]
        neg_mask = [not x for x in mask]
        train_data, test_data = data[mask], data[neg_mask]
    return train_data, test_data          

# Loding the data

In [10]:
def load_data(data, num_users, num_items, feedback="explicit"):
    users, items, scores = [], [], []
    inter = np.zeros((num_items, num_users)) if feedback == "explicit" else {}
    for line in data.itertuples():
        user_index, item_index = int(line[1] - 1), int(line[2] - 1)
        socre = int(line[3]) if feedback == "explicit" else 1
        users.append(user_index)
        items.append(item_index)
        scores.append(score)
        if feedback == "implicit":
            inter.setdefault(user_index, []).append(item_index)
        else:
            inter[item_index, user_index] = score
    return users, items, scores, inter

In [ ]:
def split_and_load(split_mode="seq-aware", feedback="explicit", test_ratio=0.1, batch_size=256):
    data, num_users, num_items = read_data